In [ ]:
%reload_ext autoreload
%autoreload 2


import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import pickle
from tqdm import tqdm
import scipy

import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils, size_control
from cottage_analysis.pipelines import pipeline_utils
from cottage_analysis.plotting import plotting_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, get_session_list
from v1_depth_analysis.v1_manuscript_2023 import size_control as plt_size_control
from v1_depth_analysis.v1_manuscript_2023 import common_utils as plt_common_utils

In [ ]:
VERSION = 1
SAVE_ROOT = '/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver'+str(VERSION)+'/fig_size_control/'
os.makedirs(SAVE_ROOT, exist_ok=True)

In [ ]:
# Load data
project = "hey2_3d-vision_foodres_20220101"

# Load example session
session_name='PZAH10.2d_S20230822'
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = size_control.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={'anatomical_only':3},
    recording_type="two_photon",
    protocol_base="SizeControl",
    photodiode_protocol=5,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name, flexilims_session=flexilims_session, project=None, conflicts="skip"
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)

# Load neurons_df of all sessions
session_list = ["PZAH10.2d_S20230822", "PZAH10.2f_S20230815", "PZAH10.2f_S20230907"]
flexilims_session = flz.get_flexilims_session(project)
neurons_df_all = plt_common_utils.concatenate_all_neurons_df(flexilims_session=flexilims_session, 
                                                             session_list=session_list, 
                                                             filename="neurons_df.pickle", 
                                                             cols=None, 
                                                             read_iscell=True,
                                                             verbose=True)

In [ ]:
fig = plt.figure()
# Example cells: depth cells with size tuning/without size tuning; non-depth cells with size tuning/without size tuning
ROIS = [453, 140, 448]
for iroi, roi in enumerate(ROIS):
    for iparam, param in enumerate(["depth", "size"]):
        ax = depth_selectivity.plot_depth_tuning_curve(
            fig=fig,
            neurons_df=neurons_df_example,
            trials_df=trials_df_example,
            roi=roi,
            param=param,
            use_col=f"{param}_tuning_popt_closedloop",
            rs_thr=None,
            rs_thr_max=None,
            still_only=False,
            still_time=0,
            frame_rate=15,
            plot_fit=False,
            linewidth=2,
            linecolor="k",
            fit_linecolor="r",
            closed_loop=1,
            overwrite_ax=True,
            ax=None,
            plot_x=0+iparam*0.25,
            plot_y=1-iroi*0.25,
            plot_width=0.15,
            plot_height=0.15,
            label=None,
            fontsize_dict={"title": 15, "label": 10, "tick": 10},
        )
        if iparam==0:
            ylim = ax.get_ylim()
        if iparam==1:
            ax.set_ylim([ylim[0]-0.05, ylim[1]+0.05])
        if iroi != len(ROIS) - 1:
            ax.set_xlabel("")

# Depth fit vs. Size fit cross-validated r2 + paired stats
plt_size_control.plot_depth_size_fit_comparison(
    fig=fig,
    neurons_df=neurons_df_all,
    filter=(neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05),
    use_cols={
        "depth_fit_r2": "depth_tuning_test_rsq_closedloop",
        "size_fit_r2": "size_tuning_test_rsq_closedloop",
        "depth_fit_pval": "depth_tuning_test_spearmanr_pval_closedloop",
        "size_fit_pval": "size_tuning_test_spearmanr_pv,al_closedloop",
    },
    plot_type="hist",
    s=5,
    c="k",
    alpha=1,
    plot_x=0,
    plot_y=0,
    plot_width=0.3,
    plot_height=0.3,
    fontsize_dict={"title": 15, "label": 10, "tick": 10},
)
    


    

    

In [ ]:
neurons_df_all["rsq_diff"] = neurons_df_all["depth_tuning_test_rsq_closedloop"] - neurons_df_all["size_tuning_test_rsq_closedloop"]
neurons_df_all.sort_values("rsq_diff", ascending=False, inplace=True)
neurons_df_all[:50]